# Launch ComfyUI with TensorRT on Brev!

Welcome!

In this notebook, we'll walk through how to use ComfyUI with TensorRT. ComfyUI is a powerful GUI for Stable Diffusion models. It uses a node-based architecture that allows users to construct very complex image/video generation workflows by connecting different nodes, each representing a different function or operation!

TensorRT engines contain state-of-the-art optimizations to performance efficiently on NVIDIA GPUs. Once the TensorRT engine is built, you can implement it in your ComfyUI workflow to take advantage of the ultra-fast performance on your RTX GPU!

In [1]:
# Environment Setup

from pathlib import Path

OPTIONS = {}

WORKSPACE = 'ComfyUI'
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

In [2]:
%%bash
if ! [ -d WORKSPACE ]; then
    echo "Starting initial setup of ComfyUI..."
    git clone https://github.com/comfyanonymous/ComfyUI
    if [ -d ComfyUI ]; then
        echo "Repository cloned successfully into 'ComfyUI'."
        cd ComfyUI
    else
        echo "Failed to clone repository."
    fi
else
    echo "Workspace directory already exists."
fi

Starting initial setup of ComfyUI...


Cloning into 'ComfyUI'...


Repository cloned successfully into 'ComfyUI'.


In [41]:
if OPTIONS['UPDATE_COMFY_UI']:
    !echo "Updating ComfyUI..."
    !git pull

!echo "Installing dependencies..."
!pip3 install accelerate
!pip3 install einops transformers>=4.25.1 safetensors>=0.3.0 aiohttp pyyaml Pillow scipy tqdm psutil
!pip3 install torch==2.2.2 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip3 install torchsde
!pip3 install kornia>=0.7.1 spandrel
!pip3 install typer

Updating ComfyUI...
Already up to date.
Installing dependencies...

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
zsh:1: 4.25.1 not found
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.3/757.3 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 9.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 MB 10.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 62.5 MB/s eta 0:00:00:00:01
  Attempting uninstall: triton
    Found existing installation: triton 2.3.0
    Uninstalling triton-2.3.0:
      Successfully uninstalled triton-2.3.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.20.5
    Uninstalling nvidia-nccl-cu12-2.20.5:


In [18]:
import os

if not os.path.isfile("ComfyUI-Manager/check.sh"):
    !chmod 755 ComfyUI-Manager/check.sh
if not os.path.isfile("ComfyUI-Manager/scan.sh"):
    !chmod 755 ComfyUI-Manager/scan.sh
if not os.path.isfile("ComfyUI-Manager/node_db/dev/scan.sh"):
    !chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
if not os.path.isfile("ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh"):
    !chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
if not os.path.isfile("ComfyUI-Manager/scripts/install-comfyui-venv-win.bat"):
    !chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat

if not os.path.isdir("ComfyUI-Manager"):
    print("-= Initial setup ComfyUI-Manager =-")
    !git clone https://github.com/ltdrdata/ComfyUI-Manager 

%cd ComfyUI-Manager
!git pull

%cd ..

%cd $WORKSPACE

chmod: cannot access 'ComfyUI-Manager/check.sh': No such file or directory
chmod: cannot access 'ComfyUI-Manager/scan.sh': No such file or directory
chmod: cannot access 'ComfyUI-Manager/node_db/dev/scan.sh': No such file or directory
chmod: cannot access 'ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh': No such file or directory
chmod: cannot access 'ComfyUI-Manager/scripts/install-comfyui-venv-win.bat': No such file or directory
-= Initial setup ComfyUI-Manager =-
Cloning into 'ComfyUI-Manager'...
remote: Enumerating objects: 10045, done.
remote: Counting objects: 100% (287/287), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 10045 (delta 206), reused 234 (delta 174), pack-reused 9758
Receiving objects: 100% (10045/10045), 6.96 MiB | 32.86 MiB/s, done.
Resolving deltas: 100% (7272/7272), done.
/home/ubuntu/verb-workspace/ComfyUI/custom_nodes/ComfyUI-Manager
Already up to date.
/home/ubuntu/verb-workspace/ComfyUI/custom_nodes
[Errno 2] No such file or di

In [22]:

if 'INSTALL_CUSTOM_NODES_DEPENDENCIES' in OPTIONS and OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
    print("-= Install custom nodes dependencies =-")
    !pip install GitPython
    !python /home/ubuntu/verb-workspace/ComfyUI/custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies

-= Install custom nodes dependencies =-

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

WARN: The `COMFYUI_PATH` environment variable is not set. Assuming 
`custom_nodes/ComfyUI-Manager/../../` as the ComfyUI path.
--------------------------------------------------------------------------------
--------------------
Restoring [1/1]: 
/home/ubuntu/verb-workspace/ComfyUI/custom_nodes/ComfyUI-Manager

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
...

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/354.4 kB[0m
[31m?[0m eta [36m-:--:--[0m
[2K     [91m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m[90m╺[0m[90m━━[0m 
[32m32

In [5]:
import os
!pip install huggingface_hub


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [6]:
!wget -c https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -P ./models/checkpoints/

--2024-06-03 23:11:30--  https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt
Resolving huggingface.co (huggingface.co)... 3.161.213.58, 3.161.213.110, 3.161.213.25, ...
Connecting to huggingface.co (huggingface.co)|3.161.213.58|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/6b/20/6b201da5f0f5c60524535ebb7deac2eef68605655d3bbacfee9cce0087f3b3f5/cc6cb27103417325ff94f52b7a5d2dde45a7515b25c255d8e396c90014281516?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27v1-5-pruned-emaonly.ckpt%3B+filename%3D%22v1-5-pruned-emaonly.ckpt%22%3B&Expires=1717715251&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzcxNTI1MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy82Yi8yMC82YjIwMWRhNWYwZjVjNjA1MjQ1MzVlYmI3ZGVhYzJlZWY2ODYwNTY1NWQzYmJhY2ZlZTljY2UwMDg3ZjNiM2Y1L2NjNmNiMjcxMDM0MTczMjVmZjk0ZjUyYjdhNWQyZGRlNDVhNzUxNWIyNWM

In [7]:
!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/

--2024-06-03 23:13:10--  https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors
Resolving huggingface.co (huggingface.co)... 3.161.213.110, 3.161.213.58, 3.161.213.25, ...
Connecting to huggingface.co (huggingface.co)|3.161.213.110|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/ec/ee/eceee26c5834d8a75cf04eeb17dfc06d1d5fe1d80c2f19520b148c11e2e98c45/735e4c3a447a3255760d7f86845f09f937809baa529c17370d83e4c3758f3c75?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27vae-ft-mse-840000-ema-pruned.safetensors%3B+filename%3D%22vae-ft-mse-840000-ema-pruned.safetensors%22%3B&Expires=1717715591&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzcxNTU5MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9lYy9lZS9lY2VlZTI2YzU4MzRkOGE3NWNmMDRlZWIxN2RmYzA2ZDFkNWZlMWQ4MGMyZjE5NTIwYjE0OGMxMWUyZTk4YzQ1LzczNW

In [ ]:
# We're going to build the TensorRT engine for this model (SDXL Turbo)
!wget -c https://huggingface.co/stabilityai/sdxl-turbo/blob/main/sd_xl_turbo_1.0_fp16.safetensors -P ./models/checkpoints/

In [30]:
%cd custom_nodes
!git clone https://github.com/comfyanonymous/ComfyUI_TensorRT

[Errno 2] No such file or directory: 'custom_nodes'
/home/ubuntu/verb-workspace/ComfyUI/custom_nodes
fatal: destination path 'ComfyUI_TensorRT' already exists and is not an empty directory.


In [32]:
%cd ComfyUI_TensorRT
!pip install -r requirements.txt

/home/ubuntu/verb-workspace/ComfyUI/custom_nodes/ComfyUI_TensorRT

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


This next cell will pull out 2 .JSON files you'll need to:

1) Build the TensorRT Engine for SDXL Turbo Model
2) Use the engine in the ComfyUI Workflow

In [ ]:
import shutil
import os

source_folder_path = '/root/verb-workspace/ComfyUI/custom_nodes/ComfyUI_TensorRT/workflows'
# source_folder_path = '/home/ubuntu/verb-workspace/ComfyUI/custom_nodes/ComfyUI_TensorRT/workflows'                (if you run into error, try this)
destination_folder_path = '/root/verb-workspace/'
# destination_folder_path = '/home/ubuntu/verb-workspace'                (if you run into error, try this)

json_files = [
    'Create_SDXL_Turbo_TRT_Static.json',
    'Build.TRT.Engine_SDXL_Turbo_Static.json'
]

shutil.move(source_folder_path, destination_folder_path)

for json_file in json_files:
    source_json_path = os.path.join(destination_folder_path, json_file)
    destination_json_path = os.path.join('/root/verb-workspace/', json_file)
    # destination_json_path = os.path.join('/home/ubuntu/verb-workspace/', json_file)         (if you run into error, try this)
    shutil.move(source_json_path, destination_json_path)
    
    assert os.path.exists(destination_json_path), f"File move failed for {json_file}"
    print(f'File {json_file} moved to {destination_json_path}')

In [23]:
import subprocess
import threading
import time
import socket

def check_server(port):
    while True:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                print("\nServer is up and running at http://129.80.69.99:{}".format(port))
                break
            time.sleep(0.5)

threading.Thread(target=check_server, daemon=True, args=(8188,)).start()


Server is up and running at http://129.80.69.99:8188


In [ ]:
!python /root/verb-workspace/ComfyUI/main.py --listen 0.0.0.0 --port 8188 --dont-print-server --disable-xformers
# !python /home/ubuntu/verb-workspace/ComfyUI/main.py --listen 0.0.0.0 --port 8188 --dont-print-server --disable-xformers

## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2024-06-04 00:24:42.787357
** Platform: Linux
** Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
** Python executable: /home/ubuntu/.pyenv/versions/3.10.14/bin/python
** Log path: /home/ubuntu/verb-workspace/ComfyUI/custom_nodes/ComfyUI_TensorRT/comfyui.log

Prestartup times for custom nodes:
   0.3 seconds: /home/ubuntu/verb-workspace/ComfyUI/custom_nodes/ComfyUI-Manager

Total VRAM 48670 MB, total RAM 483492 MB
pytorch version: 2.2.2+cu121
Set vram state to: NORMAL_VRAM
Device: cuda:0 NVIDIA RTX A6000 : cudaMallocAsync
VAE dtype: torch.bfloat16
Using pytorch cross attention
### Loading: ComfyUI-Manager (V2.36)
### ComfyUI Revision: 2222 [809cc85a] | Released on '2024-06-02'

Import times for custom nodes:
   0.0 seconds: /home/ubuntu/verb-workspace/ComfyUI/custom_nodes/websocket_image_save.py
   0.0 seconds: /home/ubuntu/verb-workspace/ComfyUI/custom_nodes/ComfyUI-Manager
   0.1 seconds: /ho

Now download the 2 JSON files to your local computer ('Create_SDXL_Turbo_TRT_Static.json','Build.TRT.Engine_SDXL_Turbo_Static.json'). The ComfyUI server should be running, so to begin, we can use the `Load` Button to bring in the 'Build.TRT.Engine_SDXL_Turbo_Static.json' file from your laptop. 

![image.png](https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/notebooks/LoadBuildjson.png?t=2024-06-04T18%3A01%3A20.036Z)

Click `Queue Prompt` to begin the workflow. This will build the TensorRT engine (takes a couple minutes)

![image.png](https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/notebooks/BuildTRTEngine.png)

Now that the engine is built, it's time to actually run a ComfyUI workflow with it and see how fast it is yourself. Click the `Load` Button again to bring in the 'Create_SDXL_Turbo_TRT_Static.json' file. 

![image.png](https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/notebooks/loadCreatejson.png?t=2024-06-04T18%3A13%3A54.983Z)


Now, click `Queue Prompt` to run this workflow and generate images with SDXL Turbo!

![image.png](https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/notebooks/TRTComfyUIWorkflow.png)

Here was my output!

![image.png](https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/notebooks/Screenshot_2024-06-04_at_11.15.57.png?t=2024-06-04T18%3A16%3A50.463Z)

In [ ]:
def check_server(port):
    while True:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                print("\nServer is up and running at http://129.80.69.99:{}".format(port))
                break
            time.sleep(0.5)

threading.Thread(target=check_server, daemon=True, args=(8188,)).start()